In [1]:
import components
import materials
import openmc

import math

In [2]:
beryllium = materials.beryllium
my_materials = openmc.Materials([beryllium])
my_materials.export_to_xml()

In [3]:
plasma_nodes = [[10, 0], [12, -1], [13, 0], [12,2]]
vessel_nodes = [[9, 0], [12, -2], [14, 0], [12,3]]

In [4]:
reflective_lower = openmc.YPlane(
    y0=0, boundary_type='reflective').rotate([0, 0, -10])
reflective_upper = openmc.YPlane(
    y0=0, boundary_type='reflective').rotate([0, 0, 10])

In [5]:
plasma = components.Plasma(nodes=plasma_nodes, boundary_1=reflective_lower, boundary_2=reflective_upper)
vacuum_vessel = components.VacuumVessel(nodes=vessel_nodes, thickness=1, material=beryllium, boundary_1=reflective_lower, boundary_2=reflective_upper)
sol = components.SOLVacuum(plasma=plasma, vacuum_vessel=vacuum_vessel, boundary_1=reflective_lower, boundary_2=reflective_upper)
blanket = components.Blanket(vacuum_vessel = vacuum_vessel, thickness =1, material=beryllium, boundary_1=reflective_lower, boundary_2=reflective_upper)
shield = components.Shield(blanket=blanket, thickness=1, material=beryllium, boundary_1=reflective_lower, boundary_2=reflective_upper)

reactor_components = [plasma, vacuum_vessel, blanket, shield]

In [6]:
sphere = openmc.Sphere(r=25, boundary_type='vacuum')
enclosure_region = -sphere
for rc in reactor_components:
    enclosure_region = enclosure_region & ~(rc.region)
enclosure_cell = openmc.Cell(region=enclosure_region, fill=None)

In [7]:
geometry=openmc.Geometry([plasma.cell, vacuum_vessel.cell, sol.cell, blanket.cell, shield.cell, enclosure_cell])
geometry.export_to_xml()

In [8]:
settings = openmc.Settings()

settings.photon_transport = False
settings.run_mode = 'fixed source'
# source definition
source = openmc.Source()
source.particle = 'neutron'
radius = openmc.stats.Discrete([620], [1])
z_values = openmc.stats.Discrete([0], [1])
angle = openmc.stats.Uniform(a=math.radians(0), b=math.radians(360))
source.space = openmc.stats.CylindricalIndependent(
    r=radius, phi=angle, z=z_values, origin=(0., 0., 0.))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete(14.1e6, 1)

# settings' settings
# weight windows from wwinp
settings.source = source
settings.survival_biasing = False
settings.batches = 100
settings.particles = int(1e6)

settings.export_to_xml()

/home/segantin/openmc/openmc/source.py:388: FutureWarning: This class is deprecated in favor of 'IndependentSource'
  warnings.warn("This class is deprecated in favor of 'IndependentSource'", FutureWarning)
